In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

def prepare_xy_time_series(
    df: pd.DataFrame,
    date_col: str,
    target_col: str,
    horizon: int = 1,          # ทำนายล่วงหน้า k step
    drop_cols: list[str] | None = None
):
    """
    สร้าง y = target ในอนาคต (t + horizon) และ X = features ณ เวลา t
    """
    df = df.sort_values(date_col).reset_index(drop=True).copy()

    # สร้าง label อนาคต
    df["__y__"] = df[target_col].shift(-horizon)

    # drop แถวท้ายที่ไม่มี y
    df = df.dropna(subset=["__y__"]).reset_index(drop=True)

    # สร้าง X
    drop_cols = (drop_cols or []) + [target_col]  # ปกติไม่ให้ใช้ target ปัจจุบันเป็น feature
    X = df.drop(columns=[c for c in drop_cols if c in df.columns] + ["__y__"])
    y = df["__y__"]

    # เก็บวันที่เผื่อ debug
    t = df[date_col].copy()

    # แนะนำ: ตรวจคอลัมน์ที่ไม่ใช่ตัวเลข
    # X = X.select_dtypes(include=[np.number])

    return X, y, t


In [14]:
def model_based_importance(model, feature_names):
    if not hasattr(model, "feature_importances_"):
        raise ValueError("Model นี้ไม่มี feature_importances_")
    imp = pd.Series(model.feature_importances_, index=feature_names).sort_values(ascending=False)
    return imp


In [15]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error

def _rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def permutation_importance_timeseries(
    model_factory,                 # ฟังก์ชันที่คืน model ใหม่ทุก fold
    X: pd.DataFrame,
    y: pd.Series,
    n_splits: int = 5,
    n_repeats: int = 10,
    scoring: str = "neg_root_mean_squared_error",  # regression: "neg_mean_absolute_error" ก็ได้
    random_state: int = 42,
):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    all_imps = []

    for fold, (tr, te) in enumerate(tscv.split(X), start=1):
        Xtr, Xte = X.iloc[tr], X.iloc[te]
        ytr, yte = y.iloc[tr], y.iloc[te]

        model = model_factory()
        model.fit(Xtr, ytr)

        r = permutation_importance(
            model, Xte, yte,
            n_repeats=n_repeats,
            random_state=random_state,
            scoring=scoring,
        )
        fold_imp = pd.Series(r.importances_mean, index=X.columns, name=f"fold{fold}")
        all_imps.append(fold_imp)

    imp_df = pd.concat(all_imps, axis=1)
    summary = pd.DataFrame({
        "importance_mean": imp_df.mean(axis=1),
        "importance_std": imp_df.std(axis=1),
    }).sort_values("importance_mean", ascending=False)

    return summary, imp_df


In [16]:
from sklearn.base import clone

def block_permutation_importance(
    model,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    metric_fn,                  # ฟังก์ชัน metric: ยิ่งน้อยยิ่งดี เช่น RMSE/MAE
    block_size: int = 24,       # ตัวอย่าง: รายชั่วโมง 24 = 1 วัน
    n_repeats: int = 20,
    random_state: int = 42
):
    rng = np.random.default_rng(random_state)

    # baseline
    base_pred = model.predict(X_test)
    base_score = metric_fn(y_test, base_pred)

    n = len(X_test)
    n_blocks = int(np.ceil(n / block_size))
    blocks = [np.arange(i*block_size, min((i+1)*block_size, n)) for i in range(n_blocks)]

    importances = {}
    for col in X_test.columns:
        deltas = []
        for _ in range(n_repeats):
            perm_blocks = blocks.copy()
            rng.shuffle(perm_blocks)

            Xp = X_test.copy()
            # เอาค่าจากบล็อกเดิม ไปวางตามลำดับบล็อกใหม่
            src_idx = np.concatenate(blocks)
            dst_idx = np.concatenate(perm_blocks)

            Xp.iloc[dst_idx, Xp.columns.get_loc(col)] = X_test.iloc[src_idx, X_test.columns.get_loc(col)].values

            pred = model.predict(Xp)
            score = metric_fn(y_test, pred)
            deltas.append(score - base_score)  # เพิ่มขึ้นเท่าไร = สำคัญเท่านั้น

        importances[col] = (float(np.mean(deltas)), float(np.std(deltas)))

    out = pd.DataFrame(importances, index=["importance_mean", "importance_std"]).T
    out = out.sort_values("importance_mean", ascending=False)
    return out, base_score


In [17]:
def drop_column_importance_timeseries(
    model_factory,
    X: pd.DataFrame,
    y: pd.Series,
    metric_fn,               # ยิ่งน้อยยิ่งดี (RMSE/MAE)
    n_splits: int = 5,
):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # ใช้ fold สุดท้ายเป็น "อนาคต" ที่ชัดสุด
    folds = list(tscv.split(X))
    tr, te = folds[-1]
    Xtr, Xte = X.iloc[tr], X.iloc[te]
    ytr, yte = y.iloc[tr], y.iloc[te]

    # baseline
    base_model = model_factory()
    base_model.fit(Xtr, ytr)
    base_score = metric_fn(yte, base_model.predict(Xte))

    results = []
    for col in X.columns:
        Xm_tr = Xtr.drop(columns=[col])
        Xm_te = Xte.drop(columns=[col])

        m = model_factory()
        m.fit(Xm_tr, ytr)
        score = metric_fn(yte, m.predict(Xm_te))

        results.append((col, score - base_score))  # คะแนนแย่ลงเท่าไร

    out = pd.DataFrame(results, columns=["feature", "importance"])
    out = out.sort_values("importance", ascending=False).reset_index(drop=True)
    return out, base_score


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

DATE_COL = "Date"         # แก้ให้ตรงไฟล์คุณ
TARGET_COL = "PM2.5"      # แก้ให้ตรงไฟล์คุณ

df = pd.read_csv("../data/cleaned_features.csv")          # หรือ path ของคุณ
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

X, y, t = prepare_xy_time_series(
    df, date_col=DATE_COL, target_col=TARGET_COL, horizon=1,
    drop_cols=[DATE_COL]  # ปกติไม่ให้ date เป็นฟีเจอร์ตรง ๆ (แต่คุณมี sin/cos แล้ว)
)

def model_factory():
    return RandomForestRegressor(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )

# 1) Permutation importance (cross-validated)
pi_summary, pi_by_fold = permutation_importance_timeseries(
    model_factory, X, y,
    n_splits=5,
    n_repeats=10,
    scoring="neg_mean_absolute_error"
)
print(pi_summary.head(20))

# 2) Block permutation บน fold สุดท้าย (สไตล์ time series)
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
tr, te = list(tscv.split(X))[-1]
m = model_factory()
m.fit(X.iloc[tr], y.iloc[tr])

block_imp, base = block_permutation_importance(
    m, X.iloc[te], y.iloc[te],
    metric_fn=lambda yt, yp: mean_absolute_error(yt, yp),
    block_size=24,    # ปรับตามความถี่ข้อมูล
    n_repeats=20
)
print("baseline MAE =", base)
block_imp
# print(block_imp.head(100))


                  importance_mean  importance_std
Vis.                     1.764773        0.952692
dayofyear_cos            0.841729        0.343480
Humi.                    0.747574        0.416430
PM2.5_lag1               0.681292        0.582687
Wind Speed               0.142028        0.146838
Pres.                    0.132519        0.094999
Prec.                    0.107205        0.138366
heatidx                  0.090300        0.070572
PM2.5_rollmean14         0.089845        0.073090
Wind Dir_cos             0.059022        0.050794
PM2.5_rollmean3          0.033012        0.045799
dayofyear_sin            0.027548        0.048981
PM2.5_rollstd14          0.021525        0.016902
PM2.5_lag14              0.020717        0.022955
PM2.5_lag3               0.016533        0.016811
Temp.                    0.016458        0.024554
PM2.5_rollmean7          0.016026        0.016947
month_cos                0.013922        0.014727
Prec._lag1               0.012106        0.011183


,importance_mean,importance_std
PM2.5_lag1,0.841348,0.339496
dayofyear_cos,0.698462,0.212616
Pres.,0.269432,0.123137
Humi.,0.263963,0.232622
Vis.,0.251170,0.336762
Wind Speed,0.232806,0.045805
PM2.5_rollmean14,0.182331,0.056814
Wind Dir_cos,0.050131,0.053319
dayofyear_sin,0.046482,0.047421
heatidx,0.043335,0.028867
